In [ ]:
import cv2                
import numpy as np      
import os             
from random import shuffle 
from tqdm import tqdm  
from sklearn.metrics import roc_auc_score    
import matplotlib.pyplot as plt 
import tensorflow as tf
import pandas as pd
import zipfile
import os, sys

In [ ]:

IMG_SIZE = 80

epochs = 5
step_size = 8
IMG_SIZE_ALEXNET = 227
validating_size = 40
nodes_fc1 = 4096
nodes_fc2 = 4096
output_classes = 2

TRAIN_DIR = os.getcwd()



print(TRAIN_DIR) 

In [ ]:

with zipfile.ZipFile("CT_COVID.zip","r") as zip_ref:
    zip_ref.extractall()

train_data = np.load(os.path.join(os.getcwd(), 'CT_COVID' ,'train_data_bi.npy'))
test_data = np.load(os.path.join(os.getcwd(), 'CT_COVID' ,'test_data_bi.npy'))

In [ ]:

for i in range(len(train_data)):
    train_data[i][0] = cv2.resize(train_data[i][0],(IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET))

for i in range(len(test_data)):
    test_data[i][0] = cv2.resize(test_data[i][0],(IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET))

train = train_data[:4000]
cv = train_data[4000:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET,3)
Y = np.array([i[1] for i in train])

cv_x = np.array([i[0] for i in cv]).reshape(-1,IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET,3)
cv_y = np.array([i[1] for i in cv])
test_x = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET,3)
test_y = np.array([i[1] for i in test_data])

In [ ]:
print(X.shape)

print(Y.shape)

print(cv_x.shape)

print(test_x.shape)

steps = len(train)
print(steps)
remaining = steps % step_size

In [ ]:

tf.reset_default_graph()

x = tf.placeholder(tf.float32,shape=[None,IMG_SIZE_ALEXNET,IMG_SIZE_ALEXNET,3])
y_true = tf.placeholder(tf.float32,shape=[None,output_classes])


In [ ]:

w_1 = tf.Variable(tf.truncated_normal([11,11,3,96], stddev=0.01))
b_1 = tf.Variable(tf.constant(0.0, shape=[[11,11,3,96][3]]))
c_1 = tf.nn.conv2d(x, w_1,strides=[1, 4, 4, 1], padding='VALID')
c_1 = c_1 + b_1
c_1 = tf.nn.relu(c_1)
print(c_1)

In [ ]:

p_1 = tf.nn.max_pool(c_1, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_1)

In [ ]:

w_2 = tf.Variable(tf.truncated_normal([5,5,96,256], stddev=0.01))

b_2 = tf.Variable(tf.constant(1.0, shape=[[5,5,96,256][3]]))
c_2 = tf.nn.conv2d(p_1, w_2,strides=[1, 1, 1, 1], padding='SAME')
c_2 = c_2 + b_2
c_2 = tf.nn.relu(c_2)
print(c_2)

In [ ]:

p_2 = tf.nn.max_pool(c_2, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_2)

In [ ]:

w_3 = tf.Variable(tf.truncated_normal([3, 3, 256, 384], stddev=0.01))

b_3 = tf.Variable(tf.constant(0.0, shape=[[3, 3, 256, 384][3]]))
c_3 = tf.nn.conv2d(p_2, w_3,strides=[1, 1, 1, 1], padding='SAME')
c_3 = c_3 + b_3
c_3 = tf.nn.relu(c_3)
print(c_3)

In [ ]:

w_4 = tf.Variable(tf.truncated_normal([3, 3, 384, 384], stddev=0.01))
b_4 = tf.Variable(tf.constant(0.0, shape=[[3, 3, 384, 384][3]]))
c_4 = tf.nn.conv2d(c_3, w_4,strides=[1, 1, 1, 1], padding='SAME')
c_4 = c_4 + b_4
c_4 = tf.nn.relu(c_4)
print(c_4)

In [ ]:

w_5 = tf.Variable(tf.truncated_normal([3, 3, 384, 256], stddev=0.01))
b_5 = tf.Variable(tf.constant(0.0, shape=[[3, 3, 384, 256][3]]))
c_5 = tf.nn.conv2d(c_4, w_5,strides=[1, 1, 1, 1], padding='SAME')
c_5 = c_5 + b_5
c_5 = tf.nn.relu(c_5)
print(c_5)

In [ ]:
p_3 = tf.nn.max_pool(c_5, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_3)

In [ ]:
flattened = tf.reshape(p_3,[-1,6*6*256])
print(flattened)

In [ ]:
input_size = int( flattened.get_shape()[1] )
w1_fc = tf.Variable(tf.truncated_normal([input_size, nodes_fc1], stddev=0.01))
b1_fc = tf.Variable( tf.constant(1.0, shape=[nodes_fc1] ) )
s_fc1 = tf.matmul(flattened, w1_fc) + b1_fc
s_fc1 = tf.nn.relu(s_fc1)

In [ ]:
hold_prob1 = tf.placeholder(tf.float32)
s_fc1 = tf.nn.dropout(s_fc1,keep_prob=hold_prob1)
print(s_fc1)

In [ ]:
w2_fc = tf.Variable(tf.truncated_normal([nodes_fc1, nodes_fc2], stddev=0.01))
b2_fc = tf.Variable( tf.constant(1.0, shape=[nodes_fc2] ) )
s_fc2 = tf.matmul(s_fc1, w2_fc) + b2_fc
s_fc2 = tf.nn.relu(s_fc2)
print(s_fc2)

In [ ]:
hold_prob2 = tf.placeholder(tf.float32)
s_fc2 = tf.nn.dropout(s_fc2,keep_prob=hold_prob1)

In [ ]:
w3_fc = tf.Variable(tf.truncated_normal([nodes_fc2,output_classes], stddev=0.01))
b3_fc = tf.Variable( tf.constant(1.0, shape=[output_classes] ) )
y_pred = tf.matmul(s_fc2, w3_fc) + b3_fc
print(y_pred)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))
train = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cross_entropy)
matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
acc = tf.reduce_mean(tf.cast(matches,tf.float32))
init = tf.global_variables_initializer()


In [ ]:
acc_list = []
auc_list = []
loss_list = []
saver = tf.train.Saver()

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'
with tf.Session(config=config) as sess:
    sess.run(init)
    for i in range(epochs):
        for j in range(0,steps-remaining,step_size):
            _,c = sess.run([train,cross_entropy],
			feed_dict={x:X[j:j+step_size] , y_true:Y[j:j+step_size],hold_prob1:0.5,hold_prob2:0.5})
        
        
		cv_auc_list = []
        cv_acc_list = []
        cv_loss_list = []
        for v in range(0,len(cv_x)-int(len(cv_x) % validating_size),validating_size):
            acc_on_cv,loss_on_cv,preds = sess.run([acc,cross_entropy,tf.nn.softmax(y_pred)],
			feed_dict={x:cv_x[v:v+validating_size] ,y_true:cv_y[v:v+validating_size] ,hold_prob1:1.0,hold_prob2:1.0})
			
            auc_on_cv = roc_auc_score(cv_y[v:v+validating_size],preds)
            cv_acc_list.append(acc_on_cv)
            cv_auc_list.append(auc_on_cv)
            cv_loss_list.append(loss_on_cv)
        acc_cv_ = round(np.mean(cv_acc_list),5)
        auc_cv_ = round(np.mean(cv_auc_list),5)
        loss_cv_ = round(np.mean(cv_loss_list),5)
        acc_list.append(acc_cv_)
        auc_list.append(auc_cv_)
        loss_list.append(loss_cv_)
        print("Epoch:",i,"Accuracy:",acc_cv_,"Loss:",loss_cv_ ,"AUC:",auc_cv_)
    
    test_auc_list = []
    test_acc_list = []
    test_loss_list = []
    for v in range(0,len(test_x)-int(len(test_x) % validating_size),validating_size):
        acc_on_test,loss_on_test,preds = sess.run([acc,cross_entropy,tf.nn.softmax(y_pred)],
		feed_dict={x:test_x[v:v+validating_size] ,y_true:test_y[v:v+validating_size] ,hold_prob1:1.0,hold_prob2:1.0})
        
        auc_on_test = roc_auc_score(test_y[v:v+validating_size],preds)
        test_acc_list.append(acc_on_test)
        test_auc_list.append(auc_on_test)
        test_loss_list.append(loss_on_test)
    saver.save(sess, os.path.join(os.getcwd(),"CNN_MC.ckpt"))
    test_acc_ = round(np.mean(test_acc_list),5)
    test_auc_ = round(np.mean(test_auc_list),5)
    test_loss_ = round(np.mean(test_loss_list),5)
    print("Test Results are below:")
    print("Accuracy:",test_acc_,"Loss:",test_loss_,"AUC:",test_auc_)

In [ ]:
f,ax=plt.subplots(1,3,figsize=(12,3))
pd.Series(acc_list).plot(kind='line',title='Accuracy on CV data',ax=ax[0])
pd.Series(loss_list).plot(kind='line',figsize=(12,7),title='Loss on CV data',ax=ax[1])
pd.Series(auc_list).plot(kind='line',figsize=(12,7),title='AUC on CV data',ax=ax[2])
plt.subplots_adjust(wspace=0.8)
ax[0].set_title('Accuracy on CV data')
ax[1].set_title('Loss on CV data')
ax[2].set_title('AUC on CV data')
plt.show()

In [ ]:
with tf.Session() as session:
    saver.restore(session, "CNN_MC.ckpt")
    print("Model restored.") 
    print('Initialized')
    k = session.run([tf.nn.softmax(y_pred)], feed_dict={x:test_x[0:64] , hold_prob1:1,hold_prob2:1})

print(np.array(k).shape)

In [ ]:
k = np.array(k).reshape(64,output_classes)

print(k[0])

pred_labels = []

for i in range(64):
    r = np.round(k[i],3).argmax()
    if r ==0 : pred_labels.append("glass")
    elif r ==1: pred_labels.append("table")

In [ ]:
w=80
h=80
columns = 8
rows = 8